Project 43


In [9]:
# Importing Libraries
import webbrowser
import time
import tweepy

# Importing directory for keys
import sys
# Windows Path
path_pass =  r'C:\Users\Gumo\Desktop\Git\Pass'
sys.path.append(path_pass)

# Import keys file
import keys

In [2]:
consumer_key = 'MnR8aP3SSKNXMam7m5AeE0TkC'
consumer_secret = 'ZDk47c0rcvwqCL4i2oaupHN7BZGhCNR53dKVC3j0mqQQBwZ6eA'
callback_url = 'oob'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret,callback_url)
redirect_url = auth.get_authorization_url()
print(redirect_url)
webbrowser.open(redirect_url)

https://api.twitter.com/oauth/authorize?oauth_token=bOZpRwAAAAABXbyQAAABffXxY_M


True

In [3]:
user_pin_input = input("Pin value: ")


In [4]:
user_pin_input

'2123060'

In [5]:
auth.get_access_token(user_pin_input)

('1309643764172947456-Kg22X6PjUH1RdifYm9Ahs4Iip4pRBC',
 'gz3xHWZ5gKmrPlrVAoFNndPY6yqyvk3peE9qrYPOvqf7c')

In [10]:
api = tweepy.API(auth)

In [12]:
me = api.me()

AttributeError: 'API' object has no attribute 'me'